# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/jrand/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jrand/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
# import os
# import getpass

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [6]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/jrand/git-repos/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/jrand/git-repos/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/jrand/git-repos/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '7690c8'. Skipping!
Property 'summary' already exists in node 'ade61a'. Skipping!
Property 'summary' already exists in node '13a1e3'. Skipping!
Property 'summary' already exists in node '23f701'. Skipping!
Property 'summary' already exists in node '48750d'. Skipping!
Property 'summary' already exists in node '011a04'. Skipping!
Property 'summary' already exists in node '3e24ad'. Skipping!
Property 'summary' already exists in node 'e67c60'. Skipping!
Property 'summary' already exists in node '5cf282'. Skipping!
Property 'summary' already exists in node '96e6f0'. Skipping!
Property 'summary' already exists in node '373f84'. Skipping!
Property 'summary' already exists in node 'bda451'. Skipping!
Property 'summary' already exists in node '9cc4f4'. Skipping!
Property 'summary' already exists in node '63b2a1'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '7690c8'. Skipping!
Property 'summary_embedding' already exists in node '13a1e3'. Skipping!
Property 'summary_embedding' already exists in node 'ade61a'. Skipping!
Property 'summary_embedding' already exists in node '23f701'. Skipping!
Property 'summary_embedding' already exists in node '373f84'. Skipping!
Property 'summary_embedding' already exists in node 'e67c60'. Skipping!
Property 'summary_embedding' already exists in node '96e6f0'. Skipping!
Property 'summary_embedding' already exists in node '011a04'. Skipping!
Property 'summary_embedding' already exists in node '3e24ad'. Skipping!
Property 'summary_embedding' already exists in node '5cf282'. Skipping!
Property 'summary_embedding' already exists in node 'bda451'. Skipping!
Property 'summary_embedding' already exists in node '48750d'. Skipping!
Property 'summary_embedding' already exists in node '63b2a1'. Skipping!
Property 'summary_embedding' already exists in node '9cc4f4'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### Answer

* SingleHopSpecificQuerySynthesizer - Generates queries that only require a single fact (single node in the KG)
* MultiHopAbstractQuerySynthesizer - "Generate conceptual, reasoning-heavy questions requiring multiple passages and some thematic synthesis."
* MultiHopSpecificQuerySynthesizer - Generates queries that require more than one fact (multiple nodes in the KG)

Finally, we can use our `TestSetGenerator` to generate our testset!

In [16]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What information is available regarding Chapte...,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context discusses Chapter 1 and c...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about?,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What is Volume 8 about in relation to clinical...,[Inclusion of Clinical Work in a Standard Term...,"Volume 8, Chapter 3 provides guidance on certa...",single_hop_specifc_query_synthesizer
3,Federal Work-Study is it a payment period prog...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,What is Appendix A about in the context of stu...,[both the credit or clock hours and the weeks ...,Appendix A is referenced as an example in the ...,single_hop_specifc_query_synthesizer
5,"How do the Pell Grant, TEACH Grant, FSEOG, and...",[<1-hop>\n\nboth the credit or clock hours and...,"The disbursement of Pell Grant, TEACH Grant, F...",multi_hop_abstract_query_synthesizer
6,How do the disbursement timing requirements di...,[<1-hop>\n\nboth the credit or clock hours and...,Disbursement timing requirements vary between ...,multi_hop_abstract_query_synthesizer
7,how do academic calendars and regulatory citat...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic calendars define the academic yea...,multi_hop_abstract_query_synthesizer
8,Volume 8 and volume 8 how does clinical work a...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that clinical work outsid...,multi_hop_specific_query_synthesizer
9,"In Appendix A and B, how do the disbursement t...",[<1-hop>\n\nboth the credit or clock hours and...,"According to the context, Appendix A explains ...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'f1f1ef'. Skipping!
Property 'summary' already exists in node '6ea8ec'. Skipping!
Property 'summary' already exists in node 'e637e4'. Skipping!
Property 'summary' already exists in node 'be71da'. Skipping!
Property 'summary' already exists in node '23b125'. Skipping!
Property 'summary' already exists in node '292792'. Skipping!
Property 'summary' already exists in node 'd5ca92'. Skipping!
Property 'summary' already exists in node '234e51'. Skipping!
Property 'summary' already exists in node '4fb4e6'. Skipping!
Property 'summary' already exists in node 'c33eec'. Skipping!
Property 'summary' already exists in node '2846b0'. Skipping!
Property 'summary' already exists in node '36f81d'. Skipping!
Property 'summary' already exists in node '8a6c06'. Skipping!
Property 'summary' already exists in node '63bde0'. Skipping!


...

Property 'summary_embedding' already exists in node 'f1f1ef'. Skipping!
Property 'summary_embedding' already exists in node '6ea8ec'. Skipping!
Property 'summary_embedding' already exists in node 'c33eec'. Skipping!
Property 'summary_embedding' already exists in node 'e637e4'. Skipping!
Property 'summary_embedding' already exists in node '234e51'. Skipping!
Property 'summary_embedding' already exists in node '63bde0'. Skipping!
Property 'summary_embedding' already exists in node '23b125'. Skipping!
Property 'summary_embedding' already exists in node '8a6c06'. Skipping!
Property 'summary_embedding' already exists in node 'd5ca92'. Skipping!
Property 'summary_embedding' already exists in node '4fb4e6'. Skipping!
Property 'summary_embedding' already exists in node '292792'. Skipping!
Property 'summary_embedding' already exists in node 'be71da'. Skipping!
Property 'summary_embedding' already exists in node '36f81d'. Skipping!
Property 'summary_embedding' already exists in node '2846b0'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Whaaat is the defenition of a Departmment in t...,"[Chapter 1 Academic Years, Academic Calendars,...",A department is a part of an academic program ...,single_hop_specifc_query_synthesizer
1,What does Chapter 3 specify regarding the incl...,[Inclusion of Clinical Work in a Standard Term...,Chapter 3 states that clinical work conducted ...,single_hop_specifc_query_synthesizer
2,FWS is it like a payment period or not and how...,[Non-Term Characteristics A program that measu...,The Federal Work-Study (FWS) Program is an exc...,single_hop_specifc_query_synthesizer
3,Could you explain the significance of the TEAC...,[both the credit or clock hours and the weeks ...,"The TEACH Grant, like other federal aid progra...",single_hop_specifc_query_synthesizer
4,How do diffrences between standard and nonstan...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work includ...,multi_hop_abstract_query_synthesizer
5,How do the conditions for including clinical w...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work conducted outsi...,multi_hop_abstract_query_synthesizer
6,How does the inclusion of clinical work in sta...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
7,"How does accelerated progression, such as comp...",[<1-hop>\n\nboth the credit or clock hours and...,Accelerated progression by completing addition...,multi_hop_abstract_query_synthesizer
8,How does the information in Volume 8 regarding...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,The information in Volume 8 explains that disb...,multi_hop_specific_query_synthesizer
9,In chapter 3 how does clinical work outside th...,[<1-hop>\n\nInclusion of Clinical Work in a St...,"According to chapter 3, clinical work conducte...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [19]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [20]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [21]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [24]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [26]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [28]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [29]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Based on the provided context, the available kinds of loans include:\n\n- Direct Subsidized Loans (available only to undergraduate students)\n- Direct Unsubsidized Loans\n- Direct PLUS Loans (including student Direct PLUS Loans and parent PLUS Loans on behalf of dependent students)\n- Subsidized and Unsubsidized Federal Stafford Loans (made under the FFEL Program before July 1, 2010, but no new loans under FFEL have been made since then)\n- Federal PLUS Loans (also under the FFEL Program before July 1, 2010)\n\nIn summary, the current loans available under the Direct Loan Program are Direct Subsidized Loans, Direct Unsubsidized Loans, and Direct PLUS Loans. The FFEL Program loans like Subsidized and Unsubsidized Federal Stafford Loans and Federal PLUS Loans are legacy loans that are no longer being issued.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [30]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [31]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

#### Answer:

- `qa_evaluator`: A canned question and answer evaluator. Apparently it uses a reference answer even though it doesn't explicitly map fields like the helpfulness evaluator. The qa one accepts a format for reference answers that matches what is already provided above with client.create_example.
- `labeled_helpfulness_evaluator`: Measures whether the response is helpful given a reference answer
- `empathy_evaluator`: Measures whether the response demonstrates empathy for the user's situation. Doesn't need a reference answer.


## LangSmith Evaluation

In [32]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'ordinary-twist-92' at:
https://smith.langchain.com/o/6848a1f0-9351-4081-bbdc-eaa03dcd1c1a/datasets/176eb702-a10b-4363-84ea-d2fe291f878a/compare?selectedSessions=441106e0-dc7b-4be2-bbb6-b2e368a78093




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the disbursement timing rules outlined ...,"Based on the provided context, the disbursemen...",None,The disbursement timing rules in Volume 2 spec...,1,1,0,5.495210,4bfa8c56-7b2d-446c-8589-127eb9f1d3ac,e3a9215f-4599-4962-a966-7a49a848ceec
1,How do the definitions of academic years and i...,The definitions of academic years and instruct...,None,Volume 2 explains that the academic year for c...,1,1,0,4.911643,53d104f6-09cc-403f-a2eb-db24acc85504,636904ea-cb42-47e1-bc66-8e26702ccb72
2,In chapter 3 how does clinical work outside th...,Based on the provided context:\n\n1. **Inclusi...,None,"According to chapter 3, clinical work conducte...",1,1,0,14.947587,ad6971a2-cbbe-40bd-9d3f-c0809ac86e2b,e4dc6681-31d5-49c0-abac-b18e5286a659
3,How does the information in Volume 8 regarding...,"Based on the provided context, accelerated pro...",None,The information in Volume 8 explains that disb...,1,1,0,5.598698,c351fb56-8376-4ebf-9a87-dd1dbe30a693,c94887ec-3fb3-4306-b6f3-0d9160253a94
4,"How does accelerated progression, such as comp...","Based on the provided context, accelerated pro...",None,Accelerated progression by completing addition...,1,1,0,5.322226,665d568c-e696-47bc-a6cf-185549cad3fa,a0281012-c1d6-4831-88cc-c3049ae73d37
5,How does the inclusion of clinical work in sta...,The inclusion of clinical work in standard ter...,None,The inclusion of clinical work in standard ter...,1,1,0,6.038090,ea852484-0df6-4b43-83ab-10de1b464b48,d4d28a79-31e6-4007-8614-518a43243bf9
6,How do the conditions for including clinical w...,"Based on the provided context, clinical work c...",None,The inclusion of clinical work conducted outsi...,1,1,0,4.810378,1c305605-d1cf-4133-9b97-43f29b904997,eebad446-aca4-4985-a95b-29a16b464f08
7,How do diffrences between standard and nonstan...,"Based on the provided context, the treatment o...",None,The context explains that clinical work includ...,1,1,0,6.614640,88c11628-d449-4082-8ce1-d1f3655dd3a8,62512bf3-5244-4d23-bcee-d57a440119a0
8,Could you explain the significance of the TEAC...,"Based on the provided context, the significanc...",None,"The TEACH Grant, like other federal aid progra...",1,1,0,6.518824,b2abd4cf-cb8f-4d32-b9cb-5c3f6752e688,c062a351-8820-409e-80bc-27d614d2023d
9,FWS is it like a payment period or not and how...,Based on the provided context:\n\n- The Federa...,None,The Federal Work-Study (FWS) Program is an exc...,1,1,0,5.425930,4a458e77-7a55-4af4-b8b8-d41fd3da8e06,70109622-9288-402e-9c99-914f94162dec


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [33]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [34]:
rag_documents = docs

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### Answer:

At the limits, a chunk size close to 0 would produce chunks too small to contain entire facts or concepts, while a chunk size close to the context window size would produce chunks so large that each one contains many facts or concepts. Ideally, each chunk contains exactly 1 fact or concept (while preserving any necessary surrounding context)

In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### Answer:

A larger embedding model should do a better job of embedding two related inputs near each other in vector space.

In [37]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [38]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [39]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [40]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question; I understand how important it is to have clear information about loan options. Based on the context provided, there are several types of loans available:\n\n1. **Direct Subsidized Loans** – These are loans where the amount you can borrow is based on your financial need, calculated as the Cost of Attendance (COA) minus your Student Aid Index (SAI) and other financial aid. Interest on these loans is typically subsidized while you’re in school.\n\n2. **Direct Unsubsidized Loans** – These loans are available regardless of financial need and can be used to supplement the Direct Subsidized Loan amount or replace other financial aid.\n\n3. **Direct PLUS Loans** – These are for parents of dependent students or for graduate/professional students to help cover the COA after other aid has been applied. There is no fixed loan limit, but the loan cannot exceed the COA minus other financial aid received. However, eligibility requirements must be met to qualify.\n\nAddit

Finally, we can evaluate the new chain on the same test set!

In [41]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'aching-tree-47' at:
https://smith.langchain.com/o/6848a1f0-9351-4081-bbdc-eaa03dcd1c1a/datasets/176eb702-a10b-4363-84ea-d2fe291f878a/compare?selectedSessions=4f6c52bb-29ec-4e10-9bda-e178eefe09f9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the disbursement timing rules outlined ...,Thank you for your thoughtful question. I unde...,None,The disbursement timing rules in Volume 2 spec...,0,0,1,6.290054,4bfa8c56-7b2d-446c-8589-127eb9f1d3ac,e3532c86-414d-48f9-a992-b7a4c26f8d14
1,How do the definitions of academic years and i...,Thank you for your thoughtful question. Based ...,None,Volume 2 explains that the academic year for c...,1,1,1,5.728856,53d104f6-09cc-403f-a2eb-db24acc85504,0e1afd33-b145-43a3-9e82-b7c8be8fc251
2,In chapter 3 how does clinical work outside th...,Thank you for your thoughtful question. It sho...,None,"According to chapter 3, clinical work conducte...",1,0,1,7.010349,ad6971a2-cbbe-40bd-9d3f-c0809ac86e2b,e683e1c9-0c87-44b2-9910-27d7a09b3538
3,How does the information in Volume 8 regarding...,Thank you for your thoughtful question. I can ...,None,The information in Volume 8 explains that disb...,1,1,1,6.589745,c351fb56-8376-4ebf-9a87-dd1dbe30a693,2ce2635b-1396-43d3-81e6-fafc7b7bdb20
4,"How does accelerated progression, such as comp...",Thank you for your thoughtful question. It sou...,None,Accelerated progression by completing addition...,1,1,1,6.559715,665d568c-e696-47bc-a6cf-185549cad3fa,144ece01-2167-4289-8084-6bd28ea601bb
5,How does the inclusion of clinical work in sta...,Thank you for your thoughtful question. It’s c...,None,The inclusion of clinical work in standard ter...,1,1,1,8.189594,ea852484-0df6-4b43-83ab-10de1b464b48,e2c86707-1d52-4a1f-b64a-24925eb6b743
6,How do the conditions for including clinical w...,Thank you for your thoughtful question—it’s cl...,None,The inclusion of clinical work conducted outsi...,1,0,1,5.116801,1c305605-d1cf-4133-9b97-43f29b904997,0e4ef871-7baa-4392-8458-9673a71c6bde
7,How do diffrences between standard and nonstan...,Thank you for your thoughtful question. I unde...,None,The context explains that clinical work includ...,1,1,1,6.342847,88c11628-d449-4082-8ce1-d1f3655dd3a8,a5263b91-351f-4416-9203-dc91b65389de
8,Could you explain the significance of the TEAC...,Thank you for your thoughtful question. I unde...,None,"The TEACH Grant, like other federal aid progra...",1,1,1,4.996541,b2abd4cf-cb8f-4d32-b9cb-5c3f6752e688,b3f0681b-92e5-4b1c-98fd-b90c81c16dc4
9,FWS is it like a payment period or not and how...,Thank you for your thoughtful question about F...,None,The Federal Work-Study (FWS) Program is an exc...,1,1,1,5.143567,4a458e77-7a55-4af4-b8b8-d41fd3da8e06,253f4c1f-608a-4611-aa9f-d40d8b240800


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.


#### Answer:

![Empathy Experiment](Empathy Experiment.png)

* The empathy metric flipped from 0 to 100% just by adding a sentence asking it to be empathetic.
* This came at a cost of lower scores on helpfulness. Adding another sentence asking it to be helpful would probably solve the issue.